## Adding Error Bounds in Python

This notebook provides instructions for adding error bounds to a fitted curve.  

---

<br>

### Entering the Data

For a given test, either of the three approaches or the mystery oils, you should have a data set that looks something like this:

<br>

<center>
<img src = https://github.com/MAugspurger/Exper_Eng/raw/main/Labs_and_Sensors/Images/Example_spreadsheet_olive.PNG width = 200>
</center>

<br>

If you have a lot of data points, a spreadsheet is really the way to go.  But since our data sets are small, we'll enter the data directly into the Python environment.  We can do that by creating an *array* for both the index and the values, and using the two lists to create a Series.   Arrays are foundational data objects in the NumPy library:

In [ ]:
import numpy as np
import pandas as pd

# NOTE: Enter experimental data: Be sure to replace this data with your data!
temps = np.array([23.7, 24.8, 27.0])
vals = np.array([40.1, 38.8, 37.2])

# Put the data in a Series and put it in the order of the index (Temps)
vis_data = pd.Series(data=vals,index=temps)
vis_data.sort_index(inplace=True)
vis_data.plot(style='o', ylabel='Viscosity (cP)', xlabel='Temperature (C)');

---

<br>

### Defining the Arrhenius curve

Next we need to fit a line.  Just as we did when we calibrated the Zahn cup, we'll set up a function of the form we want (an Arrhenius curve) and then use a least squares algorithm to find the best coefficients for the fit.

<br>

Remember that we could adapt this for any type of curve: if we wanted to fit a quadratic or linear curve, we could simply replace the `arrhenius()` function with a different function!

In [ ]:
def arrhenius(mu_0,B,T_array):
    # Choose the best way to calculate the viscosity value
    mu = mu_0 * np.exp(B/(T_array+273.15))

    # Choose the appropriate return line
    return pd.Series(index = T_array,data=mu)


Remember that to use least squares, we need to define an deviation function: a function that finds the difference between the curve and the data at each point (this is written as $y_i - y_c$ in our class notes).

In [ ]:
def deviation_func(params, data):
    arrh = arrhenius(params[0],params[1],data.index)
    errors = arrh - data
    return errors

And now we can minimize the deviation between the known data and the fitted curve by calling `leastsq()`, a function that is part of the SciPy library. We'll add a first guess at the parameters, which doesn't need to be accurate but should not be too outlandish (this may take some experimentation):

In [ ]:
import scipy.optimize as spo

# Find the optimized coefficients
params = [0.01,1300]
best_params, fit_details = spo.leastsq(deviation_func, params, vis_data)
best_params

Finally, we can plot the fitted curve using the optimized parameters, and compare the fitted curve with the experimental data points:

In [ ]:
# Create the fitted curve using the optimized parameters
fitted_curve = arrhenius(best_params[0],best_params[1],vis_data.index)

# Plot the known data points as points
vis_data.plot(ylabel = 'viscosity (cP)', xlabel = 'Temperature (C)',
           title = 'Viscosity as a Function of Temperature',
                style = 'o', label='Experimental Data',legend=True);

# Plot the fitted curve as a curve
fitted_curve.plot(label='Fitted Arrhenius Curve', legend =True);

---

<br>

### Finding the Combined Uncertainty in the Data

To find the uncertainty in the fitted curve, you will need to find the standard error of the fit (see step 4 of the Zahn cup calibration process).  

<br>

$$S_{xy}= \sqrt{\frac{1}{\nu} \sum_{i=1}^{N} (y_{i}-y_{curve,i})^2}$$

<br>

Once you have the standard error of the fit, you need to find the random uncertainty at a 95% confidence level:

<br>

$$u_{rand} = t_{\nu,\%} \frac{S_{xy}}{\sqrt{N}}$$

<br>

Here is the code cell directions from Step 4 for finding the random uncertainty: if you did this in Step 4, you can just copy what you did there into this cell:


In [ ]:
# Make an array of y_i - y_c for each known time valu
error_array = vis_data - fitted_curve
error_array

# Now square each point in the error_array
#sqerr_array = ??????


# Find the sum of the squared errors
# You may need to look up how to find the sum of a Series
#sum_err =  ????????

# Find nu (assume that the order of the fit m = 2)
#nu =   ????????

# Divide sum_err by nu and take the square root
#s_xy =  ???????



Finally, you might need to combine the random uncertainty $u_{rand}$ with other uncertainties to get a total uncertainty $u_{total}$.  For instance, if you calibrated the Zahn cup and found a calibration uncertainty, you could combine the two in quadrature:

<br>

$$u_{total} = \sqrt{u_{rand}^2 + u_{calibration}^2}$$





---

<br>

### Plotting the Curve with Uncertainty Bounds

Once you have the uncertainty $u_{total}$, we want to draw a "bounds" around the fit.  This is because the uncertainty in a the fit does *not* tell us the error in each data point, but instead tells us the potential error in the fit: how far off the fit might be.  

<br>

So we want to draw our error bars not from the data points, but from the curve itself. We can do that using MatPlotLib, which is a powerful tool for plotting in Python (and in fact is the code on which our Series and DataFrame .plot() functions are built).  `fill_between` defines an upper and lower bound for the curve, and shades in the region.  `alpha` defines the transparency of the shaded region.

<br>

Just for the sake of the example, we'll set our uncertainty to an arbitrary value of $3.0 ~cP$.  *But you will need to replace this value with your calculated total uncertainty!*

In [ ]:
import matplotlib.pyplot as plt
unc = 3.0
vis_data.plot(ylabel = 'viscosity (cP)', xlabel = 'Temperature (C)',
           title = 'Viscosity as a Function of Temperature',
                style = 'o', label='Experimental Data',legend=True);
fitted_curve.plot(label='Fitted Arrhenius Curve', legend =True);
plt.fill_between(fitted_curve.index, fitted_curve.values - unc, fitted_curve.values + unc,
                 color = 'gray', alpha = 0.3);

So now that you have a model for this type of plot, you can use it to display your own viscosity data.  Yippee!

## A couple final tips

Need to add a second set of data to your plot to compare data sets?  

* Just enter the new data set in the same way that you did the first one (in the first code cell of this notebook), and add another line to the last code cell to plot that data, too.

<br>

Need to change the range of values on your x-axis or y-axis?

* Just use the "xlim" or "ylim" keyword argument in the `.plot()` lines in the code above, like this:

In [ ]:
vis_data.plot(xlim=[20,50], ylim=[30,70])